#Data preprocessing 

In [ ]:
#installing py-torch lightning and wandb
!pip install pytorch-lightning
!pip install wandb -qU

In [ ]:
#importing required libraries 
import pytorch_lightning as pl
import os
import tarfile
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.utils.data as data_utils
from torchvision import datasets, transforms
import torch
import torchvision
import torch.nn.functional as F

#Data loading and transformation

In [ ]:
convert = transforms.Compose(
    [transforms.ToTensor(),transforms.Resize((32,32)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

indices = torch.arange(10000)

trainset = torchvision.datasets.Food101(root="./data1", split='train',
                                        download=True, transform=convert)

trainset = data_utils.Subset(trainset, indices)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)

test = torchvision.datasets.Food101(root="./data1", split='test',
                                        download=True, transform=convert)

indi = torch.arange(1000)
test = data_utils.Subset(test, indi)
testloader = torch.utils.data.DataLoader(test, batch_size=32,shuffle=True)

In [ ]:
for i,j in testloader:
  print(i.size())
  break

torch.Size([32, 3, 32, 32])


In [ ]:
import os
data_dir =r'/content/data1/food-101/images/'
classes = os.listdir(data_dir)
print(len(classes))

101


In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: te11st. Use `wandb login --relogin` to force relogin


True

#A Basic CNN

In [ ]:
class CNN_basic(pl.LightningModule):
    def __init__(self):
        super(CNN_basic, self).__init__()
        self.l1 = nn.Linear(3*32*32, 101)
    def forward(self, inp):
        return torch.relu(self.l1(inp.view(inp.size(0), -1)))

#Training function

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch, steps_per_epoch=20):
  model.train()
  training_loss = 0
  training_total = 0
  training_correct = 0
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    training_loss += loss.item()
    scores, predictions = torch.max(output.data, 1)
    training_total += target.size(0)
    training_correct += int(sum(predictions == target))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  acc = round((training_correct / training_total) * 100, 2)
  print("Epoch [{}], Loss: {}, Accuracy: {}".format(epoch, training_loss/training_total, acc), end="")
  wandb.log({"Train Loss": training_loss/training_total, "Train Accuracy": acc, "Epoch": epoch})

#Testing function

In [ ]:
def test(model, device, test_loader, criterion, classes):
  model.eval()
  testing_loss = 0
  testing_total = 0
  testing_correct = 0
  sample_images = []
  with torch.no_grad():
      for data, target in test_loader:
          data, target = data.to(device), target.to(device)
          output = model(data)
          testing_loss += criterion(output, target).item()
          scores, predictions = torch.max(output.data, 1)
          testing_total += target.size(0)
          testing_correct += int(sum(predictions == target))
  acc = round((testing_correct / testing_total) * 100, 2)
  print(" Test_loss: {}, Test_accuracy: {}".format(testing_loss/testing_total, acc))
  wandb.log({"Test Loss": testing_loss/testing_total, "Test Accuracy": acc})

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#Running CNN model

In [ ]:
network = CNN_basic().to(device)
print(network)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters())

CNN_basic(
  (l1): Linear(in_features=3072, out_features=101, bias=True)
)


In [ ]:
wandb.init(project="basic_cnn", anonymous="must")
wandb.watch(network, log="all")
for epoch in range(50):  
  train(network, device, trainloader, optimizer, criterion, epoch)
  test(network, device, testloader, criterion, classes)

print("Finished Training")
wandb.finish()

Epoch [0], Loss: 0.09965128967761994, Accuracy: 15.15 Test_loss: 0.0854694058895111, Test_accuracy: 18.6
Epoch [1], Loss: 0.0799486581325531, Accuracy: 21.61 Test_loss: 0.07915878570079804, Test_accuracy: 23.4
Epoch [2], Loss: 0.07475994964838029, Accuracy: 25.87 Test_loss: 0.0851766746044159, Test_accuracy: 21.1
Epoch [3], Loss: 0.07143900337219239, Accuracy: 28.59 Test_loss: 0.0879007203578949, Test_accuracy: 15.8
Epoch [4], Loss: 0.0689386183142662, Accuracy: 30.99 Test_loss: 0.08575747895240783, Test_accuracy: 15.4
Epoch [5], Loss: 0.06690239136219024, Accuracy: 32.3 Test_loss: 0.0812409610748291, Test_accuracy: 22.4
Epoch [6], Loss: 0.06497328740358353, Accuracy: 34.82 Test_loss: 0.08595970559120178, Test_accuracy: 17.1
Epoch [7], Loss: 0.06347404396533966, Accuracy: 35.21 Test_loss: 0.08503930377960205, Test_accuracy: 19.3
Epoch [8], Loss: 0.062007504141330716, Accuracy: 37.59 Test_loss: 0.0877744460105896, Test_accuracy: 18.0
Epoch [9], Loss: 0.060858500933647156, Accuracy: 38.5

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Test Accuracy,▄▇▆▃▇▄▅▄▅█▇▇▇▃▅▄▄▄▅▇▄▅▄▄▂▃▅▅▅▃▄▂▄▄▄▄▄▂▁▂
Test Loss,▂▁▂▂▁▂▂▂▃▂▁▂▂▃▄▄▃▃▃▃▄▃▅▄▅▅▄▅▄▆▅▆▅▅▅▅▆▇█▇
Train Accuracy,▁▂▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████
Train Loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
Epoch,49
Test Accuracy,14.2
Test Loss,0.12122
Train Accuracy,57.78
Train Loss,0.04121


#All Convolutional Net

In [ ]:
# Define a convolution neural network
class Convolutinal_net(pl.LightningModule):
    def __init__(self):
        super(Convolutinal_net, self).__init__()
        self.convolutional1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(12)
        self.convolutional2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.convolutional3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.batch_norm4ch_norm3 = nn.BatchNorm2d(24)
        self.convolutional4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.batch_norm5 = nn.BatchNorm2d(24)
        n_size = self._get_conv_output((3,32,32))
        self.flatten1 = nn.Linear(n_size, 512)
        self.flatten2 = nn.Linear(512, 101)

    def _get_conv_output(self, shape):
      batch_size = 1
      input = torch.autograd.Variable(torch.rand(batch_size, *shape))
      output_feat = self._forward_features(input)
      n_size = output_feat.data.view(batch_size, -1).size(1)
      return n_size

    def _forward_features(self, x):
      x = self.pool(F.relu(self.convolutional1(x)))
      x = self.pool(F.relu(self.convolutional2(x)))
      x = self.pool(F.relu(self.convolutional3(x)))
      return x
        
    def forward(self, x):
      x = self._forward_features(x)
      x = x.view(x.size(0), -1)
      x = F.relu(self.flatten1(x))
      x = self.flatten2(x)
      return x


In [ ]:
conv = Convolutinal_net().to(device)
print(conv)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv.parameters())

Network(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=96, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=101, bias=True)
)


In [ ]:
wandb.init(project="conv", anonymous="must")
wandb.watch(conv, log="all")
for epoch in range(50):  
  train(conv, device, trainloader, optimizer, criterion, epoch)
  test(conv, device, testloader, criterion, classes)
print("Finished Training")
wandb.finish()

Epoch [0], Loss: 0.0829294184923172, Accuracy: 10.93 Test_loss: 0.07785868740081787, Test_accuracy: 17.1
Epoch [1], Loss: 0.07536808913946151, Accuracy: 19.79 Test_loss: 0.07881108665466309, Test_accuracy: 19.9
Epoch [2], Loss: 0.0710172016263008, Accuracy: 24.58 Test_loss: 0.06357484066486359, Test_accuracy: 36.7
Epoch [3], Loss: 0.06829377290010452, Accuracy: 27.52 Test_loss: 0.06785354375839234, Test_accuracy: 32.3
Epoch [4], Loss: 0.0654031068444252, Accuracy: 29.93 Test_loss: 0.05682367837429047, Test_accuracy: 43.2
Epoch [5], Loss: 0.06305856733322143, Accuracy: 33.23 Test_loss: 0.05664510846138, Test_accuracy: 43.6
Epoch [6], Loss: 0.060622320353984835, Accuracy: 36.01 Test_loss: 0.057059803903102876, Test_accuracy: 45.0
Epoch [7], Loss: 0.05877541810274124, Accuracy: 37.48 Test_loss: 0.058139634370803835, Test_accuracy: 42.6
Epoch [8], Loss: 0.05677147629261017, Accuracy: 40.59 Test_loss: 0.058670524954795834, Test_accuracy: 40.7
Epoch [9], Loss: 0.05445069574117661, Accuracy: 

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Test Accuracy,▁▂▅▄▇▇▆▆▆█▆▇▇▆▇▆▇▇▅▅▆▆▅▆▆▅▆▅▅▅▅▅▅▆▅▅▆▅▅▆
Test Loss,▂▂▂▂▁▁▁▁▂▁▁▁▁▂▂▂▂▂▂▃▃▃▄▃▄▅▄▅▅▆▆▆▇▅▇▇▇▇██
Train Accuracy,▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█▇███████████
Train Loss,█▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Epoch,49
Test Accuracy,38.9
Test Loss,0.20525
Train Accuracy,97.22
Train Loss,0.00262


##All Convolutional Net with regularization

In [ ]:
class regularised_cnn(pl.LightningModule):
  def __init__(self):
    super(regularised_cnn, self).__init__()
    self.convolutional1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
    self.batch_norm1 = nn.BatchNorm2d(12)
    self.convolutional2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
    self.batch_norm2 = nn.BatchNorm2d(12)
    self.pool = nn.MaxPool2d(2,2)
    self.convolutional3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
    self.batch_norm3 = nn.BatchNorm2d(24)
    self.convolutional4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
    self.batch_norm5 = nn.BatchNorm2d(24)
    n_size = self._get_conv_output((3,32,32))
    self.flatten1 = nn.Linear(n_size, 512)
    self.flatten2 = nn.Linear(512, 101)
    self.dropout = nn.Dropout(0.25)

  def _get_conv_output(self, shape):
    batch_size = 1
    input = torch.autograd.Variable(torch.rand(batch_size, *shape))
    output_feat = self._forward_features(input)
    n_size = output_feat.data.view(batch_size, -1).size(1)
    return n_size

  def _forward_features(self, x):
    x = self.pool(F.relu(self.convolutional1(x)))
    x = self.pool(F.relu(self.convolutional2(x)))
    x = self.pool(F.relu(self.convolutional3(x)))
    return x
      
  def forward(self, x):
    x = self._forward_features(x)
    x = x.view(x.size(0), -1)
    x = self.dropout(x)
    x = F.relu(self.flatten1(x))
    x = self.dropout(x)
    x = self.flatten2(x)
    return x

In [ ]:
regularised_cnn = regularised_cnn().to(device)
print(regularised_cnn)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(regularised_cnn.parameters())

regularised_cnn(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=96, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=101, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [ ]:
wandb.init(project="regularised_cnn", anonymous="must")
wandb.watch(regularised_cnn, log="all")

for epoch in range(50):  
  train(regularised_cnn, device, trainloader, optimizer, criterion, epoch)
  test(regularised_cnn, device, testloader, criterion, classes)

print("Finished Training")
wandb.finish()

Epoch [0], Loss: 0.08429864344596863, Accuracy: 9.27 Test_loss: 0.07733839821815491, Test_accuracy: 21.8
Epoch [1], Loss: 0.07793224468231201, Accuracy: 16.07 Test_loss: 0.0689458476305008, Test_accuracy: 31.7
Epoch [2], Loss: 0.07321700410842895, Accuracy: 21.99 Test_loss: 0.06763817167282105, Test_accuracy: 33.3
Epoch [3], Loss: 0.07018707392215728, Accuracy: 25.11 Test_loss: 0.06748385155200959, Test_accuracy: 36.0
Epoch [4], Loss: 0.06801937022209167, Accuracy: 27.89 Test_loss: 0.05836944317817688, Test_accuracy: 41.7
Epoch [5], Loss: 0.06614044271707535, Accuracy: 29.42 Test_loss: 0.0667255746126175, Test_accuracy: 33.1
Epoch [6], Loss: 0.06461295566558838, Accuracy: 31.51 Test_loss: 0.06320724618434906, Test_accuracy: 35.3
Epoch [7], Loss: 0.06355776748657227, Accuracy: 32.45 Test_loss: 0.054621081709861755, Test_accuracy: 47.9
Epoch [8], Loss: 0.06174176638126373, Accuracy: 34.05 Test_loss: 0.05475268030166626, Test_accuracy: 47.3
Epoch [9], Loss: 0.06090915642976761, Accuracy: 

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Test Accuracy,▁▃▄▄▄▄▇▇▆▅▇▆▇▇▇▇▅▆▆▇▇▇▇▆▇▇▇███▇█▇▇▆█▇▆▇▇
Test Loss,█▆▆▆▅▄▂▂▃▃▂▃▂▃▂▂▃▃▂▂▂▂▂▂▂▂▂▁▁▂▃▁▂▂▃▂▂▃▃▂
Train Accuracy,▁▂▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████
Train Loss,█▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Epoch,49
Test Accuracy,46.8
Test Loss,0.0535
Train Accuracy,53.82
Train Loss,0.04209


#Transfer Learning

In [ ]:
import torchvision.models as models

In [ ]:
class transfer_learning(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.resnet_model = models.resnet18(pretrained=True)
        linear_size = list(self.resnet_model.children())[-1].in_features
        self.resnet_model.fc = nn.Linear(linear_size, 101)

    def forward(self, X):
        return self.resnet_model(X)

    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.lr)

In [ ]:
transfer = transfer_learning().to(device)
print(transfer)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transfer.parameters())

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


transfer_learning(
  (resnet_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [ ]:
wandb.init(project="transfer_learning", anonymous="must")
wandb.watch(transfer, log="all")

for epoch in range(50):  
  train(transfer, device, trainloader, optimizer, criterion, epoch)
  test(transfer, device, testloader, criterion, classes)

print("Finished Training")
wandb.finish()

Epoch [0], Loss: 0.06906910648345947, Accuracy: 29.59 Test_loss: 0.058611322045326236, Test_accuracy: 35.6
Epoch [1], Loss: 0.05987363650798798, Accuracy: 38.0 Test_loss: 0.054698577046394345, Test_accuracy: 38.7
Epoch [2], Loss: 0.0523358836889267, Accuracy: 45.99 Test_loss: 0.05053209984302521, Test_accuracy: 44.0
Epoch [3], Loss: 0.04907348356842995, Accuracy: 49.51 Test_loss: 0.05440404164791107, Test_accuracy: 52.8
Epoch [4], Loss: 0.045754677474498746, Accuracy: 53.5 Test_loss: 0.0618340619802475, Test_accuracy: 45.8
Epoch [5], Loss: 0.04066835623383522, Accuracy: 58.83 Test_loss: 0.047113047122955325, Test_accuracy: 57.3
Epoch [6], Loss: 0.0338924761891365, Accuracy: 65.77 Test_loss: 0.069430823802948, Test_accuracy: 52.6
Epoch [7], Loss: 0.02998514212667942, Accuracy: 69.32 Test_loss: 0.060069990038871764, Test_accuracy: 50.1
Epoch [8], Loss: 0.02307533033192158, Accuracy: 76.5 Test_loss: 0.0750342059135437, Test_accuracy: 42.4
Epoch [9], Loss: 0.017809310820698736, Accuracy: 8

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Test Accuracy,▁▂▄▆█▆▆▃█▄▇▇▇▇▇▆▇▆▇▇▆▇▆▆▅▇▆▇█▇▇▇▅▆█▇▆▇▆▅
Test Loss,▂▂▁▂▁▃▂▄▂▄▄▃▄▃▄▄▃▄▃▄▅▄▆▅▅▅▅▄▄▄▄▅█▅▄▅▅▅▅▇
Train Accuracy,▁▂▃▃▄▅▅▆▇▇▇▇▇▇██████████████████████████
Train Loss,█▇▆▆▅▄▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,49
Test Accuracy,47.3
Test Loss,0.10425
Train Accuracy,97.18
Train Loss,0.00276
